In [ ]:
from typing import List, TypeVar

import numpy as np
from pathfinding.core.diagonal_movement import DiagonalMovement
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder

from simulation.loader import load_environment, TileType
from simulation.path import build_graph, find_path, plot_path, Pos

In [ ]:
env, *_ = load_environment("../data/test-map-1.txt", 1)
plot_path(env)

## Pathlib

In [ ]:
def find_path_pathlib(environment: np.ndarray, start: Pos, end: Pos) -> List[Pos]:
    # `grid` could be reused.
    # The values of the matrix signify the walking cost and a cost of zero signifies unpassable tiles.
    # Giving seats, entries and food a higher cost prevents agents from walking over seats.
    matrix = np.ones_like(environment)
    matrix[environment == TileType.WALL]  = 0
    matrix[environment == TileType.SEAT]  = 10
    matrix[environment == TileType.ENTRY] = 10
    matrix[environment == TileType.FOOD]  = 10
    grid = Grid(matrix=matrix.T)  # The coordinates for the numpy array and pathfinding are flipped.

    start = grid.node(*start)
    end = grid.node(*end)

    finder = AStarFinder(diagonal_movement=DiagonalMovement.always)
    path, runs = finder.find_path(start, end, grid)
    return path

## Networkx

In [ ]:
find_path_networkx = find_path

In [ ]:
T = TypeVar("T")
def choose(options: List[T]) -> T:
    return options[np.random.randint(len(options))]

def select_tiles(environment: np.ndarray, tile_type: TileType) -> List[Pos]:
    i, j = np.where(environment == tile_type)
    # noinspection PyTypeChecker
    return list(zip(i, j))

# Pathfinding

Agents walk from the entry to a seat, get food, walks to the seat again and then leave from an entry.

In [ ]:
def simulate_agent(environment) -> None:
    entries = select_tiles(environment, TileType.ENTRY)
    seats   = select_tiles(environment, TileType.SEAT)
    foods   = select_tiles(environment, TileType.FOOD)

    # A random choice of waypoints.
    seat = choose(seats)
    waypoints = [choose(entries), seat, choose(foods), seat, choose(entries)]
    graph = build_graph(environment)

    for a, b in zip(waypoints, waypoints[1:]):
        path = find_path(graph, a, b)
        plot_path(environment, path)

simulate_agent(env)